# Object Detection
 
Hello, this is a very minimal notebook showing a working object detection network using just ~300 lines of modelling and training code. In this lab you will code an object detection with functionality similiar to a Single Shot Detector or YOLO detector. 
 
The first steps would be to understand what anchors are. Anchors are nothing but crops of a specific size and position in the image. The job of our object detector is to classify each anchor as containing an object or not. If the anchor contains an object, we want to find out by how much our anchor needs to change in shape and position to best fit the object in it. (By how much the height, width and center x and y cordinates change.)
 
Hence, object detection is a multi task network which does classification and regression.
 
 
The final objective of completing this notebook is to generate some visualizations of the network running on images in the validation set. In addition to this, please upload the weights of your network and share the google drive link. In this notebook below:
 
**Fill this out with the Google drive link containing model weights**

[Model State Dict (weights)](https://drive.google.com/file/d/1mkImfC5TqoVAyROv4qZyzlsoA7xErTzV/view?usp=sharing)
...
 
You should upload the completed notebook to NYU Brightspace.

In [1]:
# load important modules
!git clone https://github.com/karanchahal/objectdetection-
%cd objectdetection-

fatal: destination path 'objectdetection-' already exists and is not an empty directory.
/Users/sarthak/Repos/MS-CS/Courses/Computer Vision/Assignment_2/objectdetection-


In [2]:
from extras.shapes_loader import get_shapes_loader
from extras.util import *
import torch
import matplotlib.pyplot as plt
import matplotlib
import torch.nn as nn
from extras.encoder import ResnetEncoder
import torch.nn.functional as F
from extras.anchors import get_offsets
from extras.boxes import box_iou, nms
 
trainloader, valloader = get_shapes_loader(batch_sz=4)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

ImportError: dlopen(/Users/sarthak/Repos/MS-CS/Courses/Computer Vision/cv_env/lib/python3.8/site-packages/PIL/_imaging.cpython-38-darwin.so, 0x0002): tried: '/Users/sarthak/Repos/MS-CS/Courses/Computer Vision/cv_env/lib/python3.8/site-packages/PIL/_imaging.cpython-38-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e')), '/usr/local/lib/_imaging.cpython-38-darwin.so' (no such file), '/usr/lib/_imaging.cpython-38-darwin.so' (no such file)

## Dataset

First let's introduce our dataset. We have a Pytorch dataloader which outputs images and the ground truth data for the bounding boxes. We visualize the dataset below. You dont need to write any code here. However, you should dig into the fun ction that are called to better understand the structure of the dataset. 

In [ ]:
sample, target = iter(trainloader).next()
sample = torch.stack(sample,dim=0)

In [ ]:
vis(sample)

In [ ]:
visDet(sample, target)

### Target Data for the Dataset

The labels/targets of the dataset have 2 types of data for each image. 

1. The bounding boxes which is a tensor of size (N x 4).
here is N is the number of objects in the image and 4 correspond to:

    a. top left x coordinate
    b. top left y coordinate
    c. bottom right x coordinate
    d. bottom right y coordinate

2. The second type of targets are the classification labels of size Nx1. Here N corresponds to the number of objects in the image and the 1 corresponds to the label of the object (whether it is a triangle, square or circle).


In [ ]:
print(target[1])

### Converting Target Data to Ground Truth 

Here we assign bounding boxes to the various anchor boxes in our image. Anchor boxes are rectangular chunks in the image and are spread all across the image. In object detection, we divide the image into a large number of chunks of different sizes and specify two characteristics of that chunk:

1. Whether it has an object in it or not. This is done by measuring IOU of the chunk with the bounding boxes. If the IOU is > 0.7, we say that the chunk contains the object. If it is less than 0.3, we say the chunk only contains background. If the value is somewhere in between, we say it is a bad chunk and do not use it's contribution to the loss in our backpropogation.

2. How much should the chunk change in height, width and how much should it translate to best fit the object. These offsets are 4 values, for the height, width, center x and y coordinate. You should use measurements relative to the default anchor position/scale (see below for more details on the anchors).


Anchor boxes might be explained best by this image:
![Anchor Boxes](https://lilianweng.github.io/lil-log/assets/images/SSD-framework.png)

Now, first we create the chunks or "anchors" for our image. Since our image is 128x 128 in size. In this lab we want to use a stride of 16.  Hence, we should get 192 anchors.


### Why do we get the size 192 ?

We get the size of 192 as we have a total of 192 anchors in an image of 128 x 128 while using a stride of 16 and we have 3 anchors in each position of the image. 

Let's break this down.

128/16 = 8.

Hence, our neural network should output a feature map of 8 x 8 in spatial dimensions. Each grid in this feature map is an anchor position. Each position has 3 boxes/anchors with default shapes -> size 40x40, 50x50 and 60x60, centered at each position.

Hence the total anchors are 8x8x3 = 192. 

Your first job is to write the code that creates these 192 anchors from an image of size 128x128. Each anchor is represented by the top left and bottom right x,y coordinates of the anchors in regards with a 128x128 image. 

Hence, the output of the below function should be a tensor of size [192,4]


In [ ]:
def get_gt_boxes():
  """
  Generate 192 boxes where each box is represented by :
  [top_left_x, top_left_y, bottom_right_x, bottom_right_y]

  Each anchor position should generate 3 boxes according to the scales and ratios given.

  Return this result as a numpy array of size [192,4]
  """
  stride = 16 # The stride of the final feature map is 16 (the model compresses the image from 128 x 128 to 8 x 8)
  map_sz = 128 # this is the length of height/width of the image

  scales = torch.tensor([40,50,60])
  ratios = torch.tensor([[1,1]]).view(1,2)
  img = sample[0][0]  # Get a single image

  gt_boxes = []
  for i in range(0, map_sz, stride):
    for j in range(0, map_sz, stride):
      center_x, center_y = i + stride // 2, j + stride // 2
      for k in scales[:]:
        offset = int(k) // 2
        x_top_left, y_top_left, x_bot_right, y_bot_right = center_x - offset, center_y - offset, center_x + offset, center_y + offset
        gt_boxes.append([x_top_left, y_top_left, x_bot_right, y_bot_right])

  gt_boxes = torch.Tensor(gt_boxes)
  return gt_boxes

gt_boxes = get_gt_boxes().to(device)
assert gt_boxes.size() == (192,4)

# A Pictoral Representation of our Model

![alt text](https://cdn-images-1.medium.com/max/1600/1*7heX-no7cdqllky-GwGBfQ.png)


However instead of $2k$ in the given image we just have k scores. We group triangles, squares and circles into one foreground class for this demo. Of course, we can have seperate predictions for each class but that is added complexity and is not implemented here.


Your next job is to construct the model. Most parts of the model is already constructed, your job is to add the classification and bounding box regression heads to the encoder.

In [ ]:
class ShapesModel(nn.Module):

  def __init__(self):
    super(ShapesModel, self).__init__()

    # for each grid in the feature map we have 3 anchors of sizes: 40x40, 50x50, 60x60
    num_anchors = 3

    # regular resnet 18 encoder
    self.encoder = ResnetEncoder(num_layers=18, pretrained=False)
    self.sigm = nn.Sigmoid()

    # a small conv net
    self.conv = nn.Conv2d(
        256, 256, kernel_size=3, stride=1, padding=1
    )

    # TODO: Add a Convolutional Layer to predict the class predictions. This is a head that predicts whether a chunk/anchor contains an object or not.
    self.cls_logits = nn.Conv2d(
        256, 3, kernel_size=3, stride=1, padding=1
    )

    # TODO: Add a Convolutional Layer to predict the class predictions. This is a  a head that regresses over the 4 bounding box offsets for each anchor
    self.bbox_pred = nn.Conv2d(
        256, 12, kernel_size=3, stride=1, padding=1
    )

  def permute_and_flatten(self, layer, N, A, C, H, W):
    # helper function that rearranges the input for the loss function
    layer = layer.view(N, -1, C, H, W)
    layer = layer.permute(0, 3, 4, 1, 2)
    layer = layer.reshape(N, -1, C)  # 1x192x4
    return layer

  def get_predict_regressions(self, cls_pred, box_pred):
    # helper function that gets outputs in the right shape for applying the loss
    N, AxC, H, W = cls_pred.shape
    Ax4 = box_pred.shape[1]
    A = Ax4 // 4
    C = AxC // A
    cls_pred = self.permute_and_flatten(
        cls_pred, N, A, C, H, W
    )
    
    box_pred = self.permute_and_flatten(
        box_pred, N, A, 4, H, W
    )
    return cls_pred, box_pred

  def forward(self, x):
    bt_sz = x.size(0)

    # we take the 3rd output feature map of size 8 x 8 from
    # the resnet18 encoder this means that the stride
    # is 16 as our input image is 128x128 in size.
    x = self.encoder(x)[3]

    x = F.relu(self.conv(x))
    

    cls_pred = self.sigm(self.cls_logits(x))
    box_pred = self.bbox_pred(x)

    cls_pred, box_pred = self.get_predict_regressions(cls_pred, box_pred)

    return cls_pred.squeeze(2), box_pred


In [ ]:
m = ShapesModel()
a = torch.randn(1,3,128,128) # dummy input
cls_pred, box_pred = m(a)
print('The outputs of the net are of size: ', cls_pred.size(), box_pred.size())

assert cls_pred.size() == (1,192) and box_pred.size() == (1,192,4)

Each anchor is represented by the top left and bottom right x,y coordinates of the anchors in regards with a 128x128 image. 

Next we have to assign a class label to each anchor and in case the anchor has an IOU > 0.7 with any real object assign 4 offsets to the anchor. We do this for every single image. Each image would have it's own set of anchors with different class probabilities and offsets as that depends on what objects are situated in that image. 

Your job is to calculate those values. Fill out the below function. Hint: Use the `box_iou` and `get_offsets` function.

In [ ]:
def get_bbox_gt(ex_boxes, gt_boxes, sz=128):
  '''

  INPUT:
  ex_boxes: [Nx4]: Bounding boxes in the image. Here N is the number of bounding boxes the image has
  gt_boxes: [192 x 4]: Anchor boxes of an image of size 128 x 128 with stride 16. 
  sz : 128
  OUTPUT: 
  gt_classes: [192 x 1] : Class labels for each anchor: 1 is for foreground, 0 is for background and -1 is for a bad anchor. [where IOU is between 0.3 and 0.7]
  gt_offsets: [192 x 4]: Offsets for anchor to best fit the bounding box object. 0 values for 0 and -1 class anchors.

  '''
  high_threshold = 0.7
  low_threshold = 0.3

  # Get maximum IOU for each bounding box and the corresponding index of anchor
  bbox_iou = box_iou(ex_boxes, gt_boxes)
  bbox_max_iou, indices = torch.max(bbox_iou, 0)

  # Get labels corresponding to IOUs.
  gt_classes = bbox_max_iou.view(192, 1)
  gt_classes[gt_classes >= high_threshold] = 1
  gt_classes[gt_classes <= low_threshold] = 0
  gt_classes[(gt_classes < high_threshold) & (gt_classes > low_threshold)] = -1

  # Get offsets for all anchors.
  gt_offsets = get_offsets(gt_boxes, ex_boxes[indices, :])

  # Set invalid anchor offsets to 0
#   gt_offsets[(gt_classes < 1).reshape(192), :] = torch.tensor([0.0, 0.0, 0.0, 0.0]).to(device)

  return gt_classes, gt_offsets


In [ ]:
def get_targets(target, sample):
    '''
    Input
    target => Set of bounding boxes for each image.
    Sample => Each image
    Output:
    Bounding box offsets and class labels for each anchor.
    '''

    batched_preds = []
    batched_offsets = []
    final_cls_targets = []
    final_box_offsets = []
    for t, s in zip(target, sample):
        bboxes = t['bounding_box'].to(device).float()
        class_targets, box_offsets = get_bbox_gt(bboxes, gt_boxes, sz=128)
        final_cls_targets.append(class_targets)
        final_box_offsets.append(box_offsets)
    
    final_cls_targets = torch.stack(final_cls_targets, dim=0)
    final_box_offsets = torch.stack(final_box_offsets, dim=0)


    return final_cls_targets, final_box_offsets

In [ ]:
sample, target = iter(trainloader).next()
sample = torch.stack(sample,dim=0).to(device)


class_targets, box_targets = get_targets(target, sample)
print(class_targets.size(), box_targets.size())

This is the meat of object detection right there ! The correct calculation of the anchors for an image and it's ground truth. Now that this is done, we can move on to our neural network training code.

## Loss Function

One more small kink in the puzzle of object detection is the way the classification loss is calculated. Here, the classification loss is the binary cross entropy loss. The loss is calculated only for those anchors which are background and foreground anchors NOT the negative ones. 

One more kink in this setup is that we should sample background and foreground anchors in the ratio of 3:1. This is because there are too many background anchors and the model will predict a background anchor for every anchor and still get a 90% accuracy on the classification. To prevent this, we do this sampling which is also called hard online negative sampling. However, as we have a very small amount of anchors here (192) this isn't a problem. However, for smaller strides and different scales and ratios for each position, thie number can balloon to upto 100,000. Hence, implementing hard online negative sampling is left to be optional to you.

For the bounding box regressions, use a Smooth L1 loss, this works well and prevents outliers.

Your next task is to write the code for the class_loss and bbox_loss. The classification loss should be applied to all positive (containing an object) and negative anchors (does not contain an object). The loss should not be calculated for the anchors whose IOU values are between 0.7 and 0.3.

Apply a Smooth L1 loss to regress on the bounding box offsets, (only apply this loss for anchors which contain an object !)

**Extra Credit**: Implement Hard Online Negative Sampling, however it is not necessasary. You can get this model to train without using it as the number of anchors are low for an image in this case.



In [ ]:

def class_loss(out_pred, class_targets, device="cpu"):
    # TODO return class loss
    bce = nn.BCELoss().to(device)
    targets = class_targets.view(4, 192)
    loss = torch.zeros(1).to(device)

    # Calculate loss for each image separately
    for i in range(out_pred.shape[0]):
      pred, target, filter = out_pred[i], targets[i], targets[i] != -1

      # Do not calculate loss for maps without any relevant values
      if not torch.any(filter):
        continue

      loss += bce(pred[filter], target[filter])

    return loss


def bbox_loss(out_bbox, box_targets, class_targets, device="cpu"):
    # TODO return bounding box offset loss
    sl1 = nn.SmoothL1Loss().to(device)
    class_targets = class_targets.view(4, 192)
    loss = torch.zeros(1).to(device)

    # Calculate loss for each image separately
    for i in range(out_bbox.shape[0]):
      bbox, target, filter = out_bbox[i], box_targets[i], class_targets[i] == 1

      # Do not calculate loss for maps without any relevant values
      if not torch.any(filter):
        continue

      loss += sl1(bbox[filter, :], target[filter, :])

    return loss

## Training Function. 

This is the training function, you do not need to change any code here. You can add a validation function here to verify that your model is working well. However, you do not need to submit it.

In [ ]:
def train(ep, model, trainloader, optimizer):
        total_loss = 0
        b_loss = 0
        c_loss = 0
        for i, (ims, targets) in enumerate(trainloader):
            ims = torch.stack(list(ims), dim=0).to(device)
            class_targets, box_targets = get_targets(targets, ims)
            class_targets, box_targets = class_targets.to(device), box_targets.to(device)
            out_pred, out_box = model(ims)
            loss_cls = class_loss(out_pred, class_targets, device)
            loss_bbox = bbox_loss(out_box, box_targets, class_targets, device)
            loss = loss_cls + loss_bbox

            if loss.item() != 0:
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

            total_loss += loss.item()
            c_loss += loss_cls.item()
            b_loss += loss_bbox.item()
            
        avg_c_loss = float(c_loss / len(trainloader))
        avg_b_loss = float(b_loss / len(trainloader))
        print('Trained Epoch: {} | Avg Classification Loss: {}, Bounding loss: {}'.format(ep, avg_c_loss, avg_b_loss))

In [ ]:
model = ShapesModel().to(device)
optimizer = torch.optim.SGD(
        model.parameters(),
        lr=0.1, momentum=0.9, weight_decay=1e-4)    
for ep in range(25):
    train(ep, model, trainloader, optimizer)

### Now let's visualize the predictions !

The most exciting part of any neural network model setup. First let's visualize our ground truth.

In [ ]:
sample, target = iter(valloader).next()
sample = torch.stack(sample,dim=0)

In [ ]:
visDet(sample, target)
print(target[0])

### Our Model Predictions

Note: We perform NMS upon our final predictions to clear up the output. This is done as a lot of anchors near to the actual object will fire resulting in a lot of boxes. NMS handles these multiple predictions by merging and giving a single box for a predicted object instead of multiple. This is best signified through another image:

![NMS](https://miro.medium.com/max/1000/0*WI5_K3bAbYaRyzE-.png)

Now onto our model predictions !

Your final task is to write code to visualize the model predictions on an image in the validation set. You can use the `visDet` function to help out with visualizing. Your main job would be to select what anchors (after applying offsets) to draw on the image. Generally, if the probability is > 0.7. It's a good idea to use the anchor. Also, you can use the `nms` function inside `boxes.py`.


In [ ]:
def get_coordinates(gt_boxes, offsets, device="cpu"):
    """
    Return box corner coordinates for given top-left coordinates and offset values.

    Args:
      gt_boxes: Ground truth anchor box coordinates.
      offsets: Tensor of box top-left corner and corresponding offsets.

    Returns:
      Tensor of coordinate pairs.
    """
    # Inverse function of get_offsets()

    gt_width = gt_boxes[:, 2] - gt_boxes[:, 0]
    gt_height = gt_boxes[:, 3] - gt_boxes[:, 1]
    gt_center_x = gt_boxes[:, 0] + 0.5*gt_width
    gt_center_y = gt_boxes[:, 1] + 0.5*gt_height

    img_boxes = []
    for i in range(offsets.shape[0]):
      delta_x, delta_y, delta_scaleX, delta_scaleY = offsets[i, :, 0], offsets[i, :, 1], offsets[i, :, 2], offsets[i, :, 3]
      bbox_width = gt_width / torch.exp(delta_scaleX)
      bbox_height = gt_height / torch.exp(delta_scaleY)

      bbox_center_x = gt_center_x - delta_x * bbox_width
      bbox_center_y = gt_center_y - delta_y * bbox_height

      # Initialize empty coordinates matrix
      bbox_coord = torch.zeros(192, 4).to(device)

      bbox_coord[:, 0] = bbox_center_x - 0.5*bbox_width
      bbox_coord[:, 1] = bbox_center_y - 0.5*bbox_height
      bbox_coord[:, 2] = bbox_width + bbox_coord[:, 0]
      bbox_coord[:, 3] = bbox_height + bbox_coord[: ,1]

      # Append coordinates for this image
      img_boxes.append(bbox_coord)
    
    # Stack coordinates for all images into a single tensor
    img_boxes = torch.stack(img_boxes, dim=0)

    return img_boxes


def get_valid_boxes(box_labels, bboxes):
  image_boxes = []
  for i in range(box_labels.shape[0]):
    box_dict = {}
    
    labels, coords = box_labels[i, :], bboxes[i, :]
    filter = labels > 0.7
    labels, coords = labels[filter], coords[filter, :]


    box_dict["bounding_box"] = coords.detach().cpu()
    box_dict["labels"] = labels.detach().cpu()
    image_boxes.append(box_dict)
    
  for i in range(len(image_boxes)):
    boxes = image_boxes[i]["bounding_box"]
    scores = image_boxes[i]["labels"]
    

    if boxes.nelement() == 0:
      continue
    
    keep_idx = nms(boxes, scores, 0.7)
    scores = scores[keep_idx].view(-1, 1)
    boxes = boxes[keep_idx].view(-1, 4)

    image_boxes[i] = {"bounding_box": boxes, "labels": scores}
  
  return image_boxes



def visPred(model, sample,):
    #TODO: visualize your model predictions on the sample image.
    sample = sample.to(device)
    out_pred, out_box = model(sample)
    bboxes = get_coordinates(gt_boxes, out_box, device)

    valid_boxes = get_valid_boxes(out_pred, bboxes)


    visDet(sample.cpu(), valid_boxes)

model.eval()
visPred(model, sample)

In [ ]:
torch.save(model.state_dict(), "../cv_assignment_2_obj_detection_weights.pth")

## Conclusion

Not bad, right ? This is a very barebones implementation and trained for a very short amount of epochs. However this is the meat and bones of most anchor based state of the art detectors out today. We can improve accuracy by doing multiscale training by incorporating a Feature Pyramid Network but that is for you to discover :') 

## References

Some foundational/good papers on object detection. This list is non exhaustive

1. [Single Shot Detector](https://arxiv.org/abs/1512.02325)
2. [YOLO](https://arxiv.org/abs/1612.08242)
3. [Faster RCNN](https://arxiv.org/abs/1506.01497)
4. [Feature Pyramid Network](https://arxiv.org/abs/1612.03144)
5. Bonus Read: [DETR- State of the art Object detector based on Transformers](https://arxiv.org/abs/2005.12872)
